In [14]:
import torch
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.text_splitter import RecursiveCharacterTextSplitter

from typing import Any, List, Optional

## Task 3：司法大模型

## 1 环境准备

In [1]:
# 安装 streamlit
! pip install streamlit==1.24.0

Looking in indexes: https://mirrors.aliyun.com/pypi/simple


## 2 模型下载

In [2]:
# 向量模型下载
from modelscope import snapshot_download
model_dir = snapshot_download("AI-ModelScope/bge-small-zh-v1.5", cache_dir='.')

In [3]:
# 源大模型下载
from modelscope import snapshot_download
# model_dir = snapshot_download('IEITYuan/Yuan2-2B-Mars-hf', cache_dir='.')
model_dir = snapshot_download('IEITYuan/Yuan2-2B-July-hf', cache_dir='.')

## 3 模型RAG实战

In [4]:
# 导入所需的库
from typing import List
import numpy as np

import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM

In [5]:
# 定义向量模型类
class EmbeddingModel:
    """
    class for EmbeddingModel
    """

    def __init__(self, path: str) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained(path)

        self.model = AutoModel.from_pretrained(path).cuda()
        print(f'Loading EmbeddingModel from {path}.')

    def get_embeddings(self, texts: List) -> List[float]:
        """
        calculate embedding for text list
        """
        encoded_input = self.tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
        encoded_input = {k: v.cuda() for k, v in encoded_input.items()}
        with torch.no_grad():
            model_output = self.model(**encoded_input)
            sentence_embeddings = model_output[0][:, 0]
        sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings.tolist()

In [6]:
print("> Create embedding model...")
embed_model_path = './AI-ModelScope/bge-small-zh-v1___5'
embed_model = EmbeddingModel(embed_model_path)

> Create embedding model...
Loading EmbeddingModel from ./AI-ModelScope/bge-small-zh-v1___5.


In [7]:
# 定义向量库索引类
class VectorStoreIndex:
    """
    class for VectorStoreIndex
    """

    def __init__(self, doecment_path: str, embed_model: EmbeddingModel) -> None:
        self.documents = []
        for line in open(doecment_path, 'r', encoding='utf-8'):
            line = line.strip()
            self.documents.append(line)

        self.embed_model = embed_model
        self.vectors = self.embed_model.get_embeddings(self.documents)

        print(f'Loading {len(self.documents)} documents for {doecment_path}.')

    def get_similarity(self, vector1: List[float], vector2: List[float]) -> float:
        """
        calculate cosine similarity between two vectors
        """
        dot_product = np.dot(vector1, vector2)
        magnitude = np.linalg.norm(vector1) * np.linalg.norm(vector2)
        if not magnitude:
            return 0
        return dot_product / magnitude

    def query(self, question: str, k: int = 1) -> List[str]:
        question_vector = self.embed_model.get_embeddings([question])[0]
        result = np.array([self.get_similarity(question_vector, vector) for vector in self.vectors])
        return np.array(self.documents)[result.argsort()[-k:][::-1]].tolist() 

In [8]:
print("> Create index...")
doecment_path = './knowledge.txt'
index = VectorStoreIndex(doecment_path, embed_model)

> Create index...
Loading 3 documents for ./knowledge.txt.


In [10]:
question = '2024年8月11日清晨，父亲与孩子以及自行车队在容城县南拒马河右堤附近的路段骑行。该路段尚未正式通车，但已成为当地居民晨骑的热门地点。在骑行过程中，12岁孩子骑行速度一度达到37公里/小时，并在骑行中喊出“慢一点”。不幸的是，在一次变道中孩子不慎摔倒，倒在了对面的机动车道上，随即遭到对向行驶的汽车碾压。涉事汽车的行车记录仪显示，事故发生时汽车的行驶速度约为52公里/小时。请分析该案件的责任方。'
print('> Question:', question)

context = index.query(question)
print('> Context:', context)

> Question: 2024年8月11日清晨，父亲与孩子以及自行车队在容城县南拒马河右堤附近的路段骑行。该路段尚未正式通车，但已成为当地居民晨骑的热门地点。在骑行过程中，12岁孩子骑行速度一度达到37公里/小时，并在骑行中喊出“慢一点”。不幸的是，在一次变道中孩子不慎摔倒，倒在了对面的机动车道上，随即遭到对向行驶的汽车碾压。涉事汽车的行车记录仪显示，事故发生时汽车的行驶速度约为52公里/小时。请分析该案件的责任方。
> Context: ['郑州机械研究所有限公司（以下简称郑机所）的前身机械科学研究院1956年始建于北京，是原机械工业部直属一类综合研究院所，现隶属于国资委中国机械科学研究总院集团有限公司。郑机所伴随着共和国的成长一路走来，应运而生于首都，碧玉年华献中原。多次搬迁，驻地从北京经漯河再到郑州；数易其名，由机械科学研究院到漯河机械研究所再到郑州机械研究所，现为郑州机械研究所有限公司。1956～1958年应运而生：依据全国人大一届二次会议的提议和第一机械工业部的决策，1956年3月6日，第一机械工业部发文《（56）机技研究第66号》，通知“机械科学实验研究院”（后改名为“机械科学研究院”）在北京成立。1959～1968年首次创业：承担国家重大科研项目与开发任务，以及行业发展规划以及标准制定等工作，如“九大设备”的若干关键技术等。1969～1972年搬迁河南：1969年按照“战备疏散”的要求，机械科学研究院主体迁建河南漯河，成立“漯河机械研究所”；1972年因发展需要，改迁河南郑州，成立郑州机械研究所。1973～1998年二次创业：先后隶属于国家机械工业委员会、机械电子工业部、机械工业部；1981年4月罗干由铸造室主任升任副所长，同年经国务院批准具备硕士学位授予权；1985年“葛洲坝二、三江工程及其水电机组项目”荣获国家科技进步特等奖。1999～2016年发展壮大：1999年转企改制，隶属于国资委中国机械科学研究总院；2008年被河南省首批认定为“高新技术企业”；2011年获批组建新型钎焊材料与技术国家重点实验室；2014年被工信部认定为“国家技术创新示范企业”；历经十多年开发出填补国内外空白的大型齿轮齿条试验装备，完成了对三峡升船机齿条42.2万次应力循环次数的疲劳寿命试验测试；营业收入从几千万发展到近6亿；2017年至今协同发展：2017年经公司制改制

In [11]:
# 定义大语言模型类
class LLM:
    """
    class for Yuan2.0 LLM
    """

    def __init__(self, model_path: str) -> None:
        print("Creat tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, add_eos_token=False, add_bos_token=False, eos_token='<eod>')
        self.tokenizer.add_tokens(['<sep>', '<pad>', '<mask>', '<predict>', '<FIM_SUFFIX>', '<FIM_PREFIX>', '<FIM_MIDDLE>','<commit_before>','<commit_msg>','<commit_after>','<jupyter_start>','<jupyter_text>','<jupyter_code>','<jupyter_output>','<empty_output>'], special_tokens=True)

        print("Creat model...")
        self.model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, trust_remote_code=True).cuda()

        print(f'Loading Yuan2.0 model from {model_path}.')

    def generate(self, question: str, context: List):
        if context:
            prompt = f'背景：{context}\n问题：{question}\n请基于背景，回答问题。'
        else:
            prompt = question

        prompt += "<sep>"
        inputs = self.tokenizer(prompt, return_tensors="pt")["input_ids"].cuda()
        outputs = self.model.generate(inputs, do_sample=False, max_length=1024)
        output = self.tokenizer.decode(outputs[0])

        print(output.split("<sep>")[-1])

In [12]:
print("> Create Yuan2.0 LLM...")
# model_path = './IEITYuan/Yuan2-2B-Mars-hf'
model_path = './IEITYuan/Yuan2-2B-July-hf'
llm = LLM(model_path)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


> Create Yuan2.0 LLM...
Creat tokenizer...


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


Creat model...


Some weights of the model checkpoint at ./IEITYuan/Yuan2-2B-July-hf were not used when initializing YuanForCausalLM: ['model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'mod

Loading Yuan2.0 model from ./IEITYuan/Yuan2-2B-July-hf.


In [13]:
print('> Without RAG:')
llm.generate(question, [])

print('> With RAG:')
llm.generate(question, context)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


> Without RAG:
 根据提供的信息，该案件的责任方可以是以下几个方面：
1. 自行车骑行者：根据描述，12岁的孩子在骑行过程中喊出“慢一点”，这表明他可能对自行车的速度和安全有更高的要求。因此，自行车骑行者可能需要承担一定的责任。
2. 自行车骑行者与自行车队：自行车队在骑行过程中可能没有遵守交通规则，例如超速、闯红灯等，这可能导致事故的发生。因此，自行车骑行者和自行车队可能需要承担一定的责任。
3. 自行车骑行者与道路条件：道路条件也可能对事故的发生产生影响。如果道路条件不良，例如路面湿滑、坑洼不平等，自行车骑行者可能需要承担一定的责任。
4. 自行车骑行者与车辆：如果自行车骑行者在变道时不慎摔倒，并被对向行驶的汽车碾压，那么自行车骑行者可能需要承担一定的责任。
综上所述，该案件的责任方可能包括自行车骑行者、自行车队、道路条件和车辆。具体责任的分配需要根据具体情况和证据来确定。<eod>
> With RAG:
 根据提供的背景信息，无法确定该案件的责任方。<eod>
